In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
!pip install ktrain

     |████████████████████████████████| 174kB 3.4MB/s 
     |████████████████████████████████| 317kB 46.7MB/s 
     |████████████████████████████████| 112kB 40.9MB/s 
     |████████████████████████████████| 1.0MB 45.9MB/s 
     |████████████████████████████████| 204kB 45.3MB/s 
     |████████████████████████████████| 1.8MB 18.1MB/s 
  Created wheel for ktrain: filename=ktrain-0.5.2-cp36-none-any.whl size=213552 sha256=1f561797bc0992aa069cb9e78b103c89410344454293ae6959d69e1c0bf89493
  Stored in directory: /root/.cache/pip/wheels/72/71/d8/54ea825a57a816701162b9b0696765593182ce37d9f30d7819
  Created wheel for keras-bert: filename=keras_bert-0.80.0-cp36-none-any.whl size=37923 sha256=e46a03777ebf2e62a6d395b746f6bff55ec8f34b0b289cb6a2d81026c1dc1e94
  Stored in directory: /root/.cache/pip/wheels/63/dc/87/3260cb91f3aa32c0f85c5375429a30c8fd988bbb48f5ee21b0
  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7424 sha256=d13f2ce8ea6ccb508e6c87b325a23840b736a09b0eba265aa20

In [3]:
!git clone https://github.com/elnagara/HARD-Arabic-Dataset.git

Cloning into 'HARD-Arabic-Dataset'...
remote: Enumerating objects: 100, done.
remote: Total 100 (delta 0), reused 0 (delta 0), pack-reused 100
Receiving objects: 100% (100/100), 116.36 MiB | 46.78 MiB/s, done.
Resolving deltas: 100% (35/35), done.


In [4]:
cd /content/HARD-Arabic-Dataset/data

/content/HARD-Arabic-Dataset/data


In [6]:
!unzip balanced-reviews.zip

Archive:  balanced-reviews.zip
  inflating: balanced-reviews.txt    


In [0]:
data=pd.read_csv('/content/HARD-Arabic-Dataset/data/balanced-reviews.txt', delimiter='\t', encoding='utf-16')

In [39]:
data.head()

,no,Hotel name,rating,user type,room type,nights,review
0,2,فندق 72,2,مسافر منفرد,غرفة ديلوكس مزدوجة أو توأم,أقمت ليلة واحدة,“ممتاز”. النظافة والطاقم متعاون.
1,3,فندق 72,5,زوج,غرفة ديلوكس مزدوجة أو توأم,أقمت ليلة واحدة,استثنائي. سهولة إنهاء المعاملة في الاستقبال. ل...
2,16,فندق 72,5,زوج,-,أقمت ليلتين,استثنائي. انصح بأختيار الاسويت و بالاخص غرفه ر...
3,20,فندق 72,1,زوج,غرفة قياسية مزدوجة,أقمت ليلة واحدة,“استغرب تقييم الفندق كخمس نجوم”. لا شي. يستحق ...
4,23,فندق 72,4,زوج,غرفة ديلوكس مزدوجة أو توأم,أقمت ليلتين,جيد. المكان جميل وهاديء. كل شي جيد ونظيف بس كا...


In [0]:
data['rating']= data['rating'].apply(lambda x: 'pos' if x>3 else 'neg')

In [41]:
data.head()

,no,Hotel name,rating,user type,room type,nights,review
0,2,فندق 72,neg,مسافر منفرد,غرفة ديلوكس مزدوجة أو توأم,أقمت ليلة واحدة,“ممتاز”. النظافة والطاقم متعاون.
1,3,فندق 72,pos,زوج,غرفة ديلوكس مزدوجة أو توأم,أقمت ليلة واحدة,استثنائي. سهولة إنهاء المعاملة في الاستقبال. ل...
2,16,فندق 72,pos,زوج,-,أقمت ليلتين,استثنائي. انصح بأختيار الاسويت و بالاخص غرفه ر...
3,20,فندق 72,neg,زوج,غرفة قياسية مزدوجة,أقمت ليلة واحدة,“استغرب تقييم الفندق كخمس نجوم”. لا شي. يستحق ...
4,23,فندق 72,pos,زوج,غرفة ديلوكس مزدوجة أو توأم,أقمت ليلتين,جيد. المكان جميل وهاديء. كل شي جيد ونظيف بس كا...


In [0]:
df=data[['review', 'rating']]

In [43]:
df.head()

,review,rating
0,“ممتاز”. النظافة والطاقم متعاون.,neg
1,استثنائي. سهولة إنهاء المعاملة في الاستقبال. ل...,pos
2,استثنائي. انصح بأختيار الاسويت و بالاخص غرفه ر...,pos
3,“استغرب تقييم الفندق كخمس نجوم”. لا شي. يستحق ...,neg
4,جيد. المكان جميل وهاديء. كل شي جيد ونظيف بس كا...,pos


In [0]:
dummy=pd.get_dummies(df['rating'])

In [46]:
dummy.head()

,neg,pos
0,1,0
1,0,1
2,0,1
3,1,0
4,0,1


In [47]:
df=pd.concat([df,dummy], axis=1)
df.head()

,review,rating,neg,pos
0,“ممتاز”. النظافة والطاقم متعاون.,neg,1,0
1,استثنائي. سهولة إنهاء المعاملة في الاستقبال. ل...,pos,0,1
2,استثنائي. انصح بأختيار الاسويت و بالاخص غرفه ر...,pos,0,1
3,“استغرب تقييم الفندق كخمس نجوم”. لا شي. يستحق ...,neg,1,0
4,جيد. المكان جميل وهاديء. كل شي جيد ونظيف بس كا...,pos,0,1


In [0]:
df=df.drop('rating', axis=1)

In [49]:
df.head()

,review,neg,pos
0,“ممتاز”. النظافة والطاقم متعاون.,1,0
1,استثنائي. سهولة إنهاء المعاملة في الاستقبال. ل...,0,1
2,استثنائي. انصح بأختيار الاسويت و بالاخص غرفه ر...,0,1
3,“استغرب تقييم الفندق كخمس نجوم”. لا شي. يستحق ...,1,0
4,جيد. المكان جميل وهاديء. كل شي جيد ونظيف بس كا...,0,1


In [0]:
import ktrain
from ktrain import text

In [51]:
(X_train,Y_train),(X_test,Y_test), preproc = text.texts_from_df(df,'review',label_columns=['neg','pos'],maxlen=75, 
                                                                   max_features=100000,
                                                                   preprocess_mode='standard',
                                                                   val_pct=0.1,
                                                                   ngram_range=3)

language: ar
Word Counts: 151670
Nrows: 95128
95128 train sequences
Average train sequence length: 23
Adding 3-gram features
max_features changed to 2686776 with addition of ngrams
Average train sequence length with ngrams: 66
x_train shape: (95128,75)
y_train shape: (95128,2)
10570 test sequences
Average test sequence length: 22
Average test sequence length with ngrams: 41
x_test shape: (10570,75)
y_test shape: (10570,2)


In [53]:
model=text.text_classifier('nbsvm',(X_train,Y_train), preproc=preproc)

Is Multi-Label? False
compiling word ID features...
maxlen is 75
building document-term matrix... this may take a few moments...
rows: 1-10000
rows: 10001-20000
rows: 20001-30000
rows: 30001-40000
rows: 40001-50000
rows: 50001-60000
rows: 60001-70000
rows: 70001-80000
rows: 80001-90000
rows: 90001-95128
computing log-count ratios...
done.


In [0]:
learner=ktrain.get_learner(model,train_data=(X_train,Y_train), val_data=(X_test,Y_test), batch_size=32)

In [56]:
learner.autofit(5e-3,1)



begin training using triangular learning rate policy with max lr of 0.005...
Train on 95128 samples, validate on 10570 samples
Epoch 1/1
95128/95128 [==============================] - 135s 1ms/step - loss: 0.2196 - acc: 0.9357 - val_loss: 0.1695 - val_acc: 0.9424


In [0]:
p=ktrain.get_predictor(learner.model,preproc)

In [59]:
p.predict("الغرفة كانت نظيفة ، الطعام ممتاز ، وأنا أحب المنظر من غرفتي.")

'pos'

In [60]:
p.predict('كان هذا الفندق باهظ الثمن والموظفين غير مهذبين.')


'neg'